# Model Experiments - Qwen Messaging Agent

This notebook provides tools for experimenting with different model configurations, hyperparameters, and evaluation metrics.

## What you'll experiment with:
- Model fine-tuning parameters
- Hyperparameter optimization
- A/B testing different model versions
- Performance evaluation and comparison
- Response quality assessment


In [ ]:
# Import libraries for model experimentation
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Tuple
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Google Cloud imports
from google.cloud import aiplatform
from google.cloud import storage

# Local imports
import sys
sys.path.append('../')
from ml.ab_testing import ABTestingFramework
from quality.quality_checks import QualityChecker

print("🧪 Model experimentation libraries loaded successfully!")


In [ ]:
# Configuration
PROJECT_ID = os.getenv('PROJECT_ID', 'your-project-id')
REGION = os.getenv('REGION', 'us-central1')

# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

# Initialize quality checker
quality_checker = QualityChecker()

print(f"🚀 Initialized experimentation environment for project: {PROJECT_ID}")
print(f"📍 Region: {REGION}")


In [ ]:
# Test conversation scenarios for evaluation
test_scenarios = [
    {
        "scenario": "Ticket Purchase",
        "message": "I need 2 tickets for tonight's Lakers game",
        "expected_tools": ["get_event_info", "check_inventory"],
        "expected_response_type": "helpful"
    },
    {
        "scenario": "Seat Upgrade",
        "message": "Can I upgrade my seats to a better section?",
        "expected_tools": ["upgrade_tickets"],
        "expected_response_type": "helpful"
    },
    {
        "scenario": "General Inquiry",
        "message": "What time does the game start?",
        "expected_tools": ["get_event_info"],
        "expected_response_type": "informative"
    },
    {
        "scenario": "Complex Request",
        "message": "I want to buy 4 tickets in section A, but if that's not available, I'll take section B, and I also want to know about parking options",
        "expected_tools": ["get_event_info", "check_inventory"],
        "expected_response_type": "comprehensive"
    }
]

print(f"📝 Loaded {len(test_scenarios)} test scenarios for evaluation")


In [ ]:
# Model evaluation function
def evaluate_model_performance(endpoint_id: str, test_scenarios: List[Dict]) -> Dict[str, Any]:
    """Evaluate model performance on test scenarios"""
    
    results = {
        "endpoint_id": endpoint_id,
        "timestamp": datetime.now().isoformat(),
        "scenarios": [],
        "overall_metrics": {}
    }
    
    # Initialize endpoint
    endpoint = aiplatform.Endpoint(f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_id}")
    
    for scenario in test_scenarios:
        print(f"🧪 Testing scenario: {scenario['scenario']}")
        
        # Get model response
        start_time = datetime.now()
        response = endpoint.predict(instances=[{"messages": [{"role": "user", "content": scenario["message"]}]}])
        end_time = datetime.now()
        
        response_time = (end_time - start_time).total_seconds() * 1000
        model_response = response.predictions[0]["response"]
        
        # Quality assessment
        quality_scores = quality_checker.assess_quality(
            user_message=scenario["message"],
            agent_response=model_response
        )
        
        scenario_result = {
            "scenario": scenario["scenario"],
            "message": scenario["message"],
            "response": model_response,
            "response_time_ms": response_time,
            "quality_scores": quality_scores,
            "expected_tools": scenario["expected_tools"],
            "expected_response_type": scenario["expected_response_type"]
        }
        
        results["scenarios"].append(scenario_result)
    
    # Calculate overall metrics
    response_times = [s["response_time_ms"] for s in results["scenarios"]]
    quality_scores = [s["quality_scores"]["overall_score"] for s in results["scenarios"]]
    
    results["overall_metrics"] = {
        "avg_response_time_ms": np.mean(response_times),
        "avg_quality_score": np.mean(quality_scores),
        "total_scenarios": len(test_scenarios)
    }
    
    return results

print("✅ Model evaluation function defined")
